<a href="https://colab.research.google.com/github/abdibrokhim/BrAIn-MRI-App-API/blob/main/brainmri_google_gemma_finetuning_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 29.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer

In [5]:
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [6]:
model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [8]:
text = "Quote: Our doubts are traitors,"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Our doubts are traitors, and make us lose the good we oft might win, by fearing to attempt.

-William Shakespeare


In [9]:
os.environ["WANDB_DISABLED"] = "false"

In [10]:
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM",
)

In [11]:
from datasets import load_dataset

# Load the JSONL file
data = load_dataset("json", data_files="/content/training.jsonl")

# Map the data
data = data.map(lambda samples: tokenizer(samples["prompt"], samples["completion"]), batched=True)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/220 [00:00<?, ? examples/s]

In [12]:
data

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion', 'input_ids', 'attention_mask'],
        num_rows: 220
    })
})

In [13]:
def formatting_func(example):
    text = f"Prompt: {example['prompt'][0]}\Write Conclusion: {example['completion'][0]}"
    return [text]

In [14]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=16,
        gradient_accumulation_steps=16,
        warmup_steps=100,
        max_steps=1000,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=20,
        output_dir="outputs",
        optim="adamw_torch"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/220 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [15]:
trainer.train()

Step,Training Loss
20,0.148100
40,0.118500
60,0.028300
80,0.001400
100,0.000500
120,0.000000
140,0.000000
160,0.000000
180,0.000000
200,0.000000


TrainOutput(global_step=1000, training_loss=0.005937702980823815, metrics={'train_runtime': 767.2409, 'train_samples_per_second': 333.663, 'train_steps_per_second': 1.303, 'total_flos': 7193985146880000.0, 'train_loss': 0.005937702980823815, 'epoch': 1000.0})

In [17]:
text = """
Question:
Scanning technique: T1 FSE-sagital, T2 FLAIR, T2 FSE-axial, T2 FSE-coronar.
On a series of tomograms, in the structure of the subcortical and periventricular white matter of the cerebral hemispheres on both sides, single round-shaped foci of pathological intensity are not symmetrically determined, with unclear contours, homogeneous hyperintense signal characteristics on T2 VI and T2 FLAIR, with a diameter of 1-3 mm.
The longitudinal fissure of the cerebrum is located centrally. Convexital grooves are not expanded, their number is not changed. The thickness of the cerebral cortex is preserved, the distribution of gray matter is not disturbed.
The lateral ventricles are symmetrical, the width of the ventricles at the level of the foramen of Monro is 5 mm on the right, 6 mm on the left. The third ventricle is 3 mm wide. Sylvian aqueduct has not been changed. The fourth ventricle is tent-shaped and not dilated. The cisterna magna is dilated with retrocerbellar extension.
The basal ganglia are usually located, symmetrical, with clear, even contours, the dimensions are not changed, the MR signal is not changed. The corpus callosum is of normal shape and size. The brain stem is without features. The cerebellum is of normal shape, the signal characteristics of the white matter are not changed. The width of the cerebellar cortex is preserved. The craniovertebral junction is unchanged.
The pituitary gland is of normal shape, height in the sagittal projection is 6 mm. The pituitary stalk is located centrally. The chiasma of the optic nerves is located usually, the contours are clear and even. Parasellar cisterns without areas of pathological intensity. The siphons of the internal carotid arteries are not changed. The cavernous sinuses of both carotid arteries are symmetrical, with clear, even contours.
The shape of the orbital cones on both sides is unchanged. The eyeballs are spherical in shape and of normal size; the MR signal of the vitreous body is not changed. The diameter of the optic nerves was preserved. The perineural subarachnoid space of the orbits is moderately diffusely dilated. The extraocular muscles are of normal size, the structure is without pathological signals. Retrobulbar fatty tissue without pathological signals.
Region of the cerebellopontine angle: the prevestocochlear nerve is clearly differentiated on both sides. Pneumatization of the cells of the mastoid processes of the temporal bones on the left is not impaired, on the right it is reduced due to sclerotic changes.
The paranasal sinuses are developed correctly. The nasal turbinates are markedly hypertrophied. There is thickening of the mucous membranes of the cells of the ethmoidal labyrinth on both sides. In the right maxillary sinus, uneven hyperplasia of the mucous membrane with the presence of inclusions is noted, with transitional signal characteristics on T1 and T2 VI; on the left, there is thickening of the mucous membrane up to 4 mm. In the main sinus on the right, a cyst measuring 11x10 mm is detected. The nasal septum is slightly curved to the left.

Write Conclusion:
"""
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Question: 
Scanning technique: T1 FSE-sagital, T2 FLAIR, T2 FSE-axial, T2 FSE-coronar.
On a series of tomograms, in the structure of the subcortical and periventricular white matter of the cerebral hemispheres on both sides, single round-shaped foci of pathological intensity are not symmetrically determined, with unclear contours, homogeneous hyperintense signal characteristics on T2 VI and T2 FLAIR, with a diameter of 1-3 mm.
The longitudinal fissure of the cerebrum is located centrally. Convexital grooves are not expanded, their number is not changed. The thickness of the cerebral cortex is preserved, the distribution of gray matter is not disturbed.
The lateral ventricles are symmetrical, the width of the ventricles at the level of the foramen of Monro is 5 mm on the right, 6 mm on the left. The third ventricle is 3 mm wide. Sylvian aqueduct has not been changed. The fourth ventricle is tent-shaped and not dilated. The cisterna magna is dilated with retrocerbellar extension.
The ba

In [18]:
# Save the fine-tuned model to a specific directory
model.save_pretrained("./gemma-fine-tuned-brainmri_2402")

In [19]:
# You can also save the tokenizer if you have one
tokenizer.save_pretrained("./gemma-fine-tuned-tokenizer-brainmri_2402")

('./gemma-fine-tuned-tokenizer-brainmri_2402/tokenizer_config.json',
 './gemma-fine-tuned-tokenizer-brainmri_2402/special_tokens_map.json',
 './gemma-fine-tuned-tokenizer-brainmri_2402/tokenizer.model',
 './gemma-fine-tuned-tokenizer-brainmri_2402/added_tokens.json',
 './gemma-fine-tuned-tokenizer-brainmri_2402/tokenizer.json')